# Learning about spatial data and maps for archaeology (and other things)

### Spatial Thinking and Skills Exercise 1 for Theory and Practice

#### Made by Rachel Opitz, Archaeology, University of Glasgow



Archaeologists regularly work with maps and data about where sites, samples and objects are found. Archaeological survey in particular relies on the collection and analysis of spatial data to understand patterns across whole areas of the landscape. Are most sites in a given landscape from a particular period? Are there more cairns than anything else? Are sites from all periods represented equally? These are the kinds of questions one might as using survey data. 


The aim of this exercise is for you to:
* learn to investigate patterns in archaeological survey data using spatial analytical tools
* learn to learn to make maps that illustrate patterns in archaeological survey data
* start thinking about the meaning of the patterns of sites and features in the landscape

You'll do this using data available from Canmore that describes the location and type of monuments surveyed throughout the Shetlands. Canmore houses survey data on sites and monuments recorded throughout Scotland.

As you may recall from Archaeology of Scotland, to start working with spatial data and maps, you need to put together your toolkit. You're currently working inside something called a jupyter notebook. It's a place to keep notes, pictures, code and maps together. You can add tools and data into your jupyter notebook and then use them to ask spatial questions and make maps and visualisations that help answer those questions. 


### Let's get started... Hit 'Ctrl'+'Enter' to run the code in any cell in the page.

[Canmore spatial data can be found here.](https://canmore.org.uk/content/data-downloads)

### We'll start by adding some of the tools we will need. They're not quite like these tools...

![They're not quite like these tools...](http://www.truelles-pixels.mom.fr/english/episode01/knowmore_files/outils.jpg)

In [2]:
%matplotlib inline
# Matplotlib is your tool for drawing graphs and basic maps. You need this!

import pandas as pd
import requests
import fiona
import geopandas as gpd
import ipywidgets as widgets
import bokeh

# These are what we call prerequisites. They are basic toosl you need to get started.
# Pandas manipulate data. Geo-pandas manipulate geographic data. They're also black and white and like to eat bamboo... 
# You need these to manipulate your data!
# Fiona helps with geographic data.
# Requests are for asking for things. It's good to be able to ask for things.
# ipywidgets supports interactivity.


# Remember to hit Ctrl+Enter to make things happen!

ImportError: No module named 'geopandas'

### Now that we have the basic tools loaded up we need to load the Canmore data. Canmore provides polygons showing the site extents. I've converted them to a set of points located at the centre of each polygon to make things a little simpler in this exercise.

In [ ]:
url = 'https://raw.githubusercontent.com/ropitz/spatialarchaeology/master/data/linlithgow_burials.geojson'
# This is where I put the data. It's in a format called geojson, used to represent geometry (shapes) and attributes (text).
request = requests.get(url)
# Please get me the data at that web address (url)
b = bytes(request.content)
# I will use the letter 'b' to refer to the data, like a nickname
with fiona.BytesCollection(b) as f:
    crs = f.crs
    linlithgow_burials = lpd.GeoDataFrame.from_features(f, crs=crs)
    print(linlithgow_burials.head())
# I will use the fiona tool to wrap up all the data from 'b', check the coordinate system (crs) listed in the features
# and print out the first few lines of the file so I can check everything looks ok. 
# Don't worry if you don't understand all the details of this part!



### Does that look right? 

You should have geometry information about polylines (the outline of the shape of each burial) of the contexts from Linlithgow and a bunch of descriptions and information about the burials archaeology. Importatly you should be able to spot the column names: Descriptio, objectid, shape_area, shape_leng, SU, definition, finds_note, formation, geometry, interpret. Each column contains a different type of information. Note that SU (stratigraphic unit) = context.

In [ ]:
# Let's visualise the data to double check that all is well

linlithgow_map1 = linlithgow_burials.plot(column='AT', cmap='Pastel2', edgecolor='grey', figsize=(10, 10));
# 'plot' means draw me an image showing the geometry of each feature in my data. 
# We want to control things like the color of different types of burials on our map. 
# I used the pastel colorscale command (cmap stands for 'colour map') 
# and asked it to draw the polygons differently based on the type of burial.


### Well done! You just made your first archaeological map. It shows all the  burials excavated at Carmelite Linlithgow. 

This is good, but what if you only want to look at one kind of burial? We can select specific types of burials from within our dataset by searching (aka querying) for them. 

How do we know what kind of burials we have? Looking at what's inside the data describing all those shapes on the map should help. 

Start by printing out our data in a tidy way. Just type its name...

In [ ]:
linlithgow_burials

In archaeology we often talk about the dates at which different things happened, and when activities started and ended. Sometimes we are not sure when something happened, so we give a range of dates with an early guess and a late guess. Look at the 'e_date' and 'l_date' columns in the table, and you'll see the date ranges guessed for each burial.

In [ ]:
# Say you only want to look at the burials from before 1400, the first ones. Pandas use square brackets [] to make selections. 
# Here we select all the rows (.loc) where the column 'L_DATE' has a value less than 1400. < means 'less than' in code

gabii_areaB.loc[linlithgow_burials['L_DATE']<1400]

In [ ]:
# If we want to see this result as a map, we just add the .plot command to the end.

linlithgow_burials.loc[linlithgow_burials['L_DATE'] <1400].plot(column='AT', cmap='OrRd', figsize=(10, 10))

In [ ]:
# Try and do the same thing for burials that are earlier than 1500
linlithgow_burials.loc[linlithgow_burials(['L_DATE']<1500 AND ['L_DATE']>1400)]

In [ ]:
# Remember to draw it as a map!
linlithgow_burials.loc[linlithgow_burials['L_DATE'] <1500].plot(column='AT', cmap='OrRd', figsize=(10, 10))

In [ ]:
# Let's save these selections of 'pre1400' and 'pre1500' so we can use them again.
# I've given them names here. These are now 'named variables'
pre1400 = linlithgow_burials.loc[linlithgow_burials['L_DATE'] <1400]
pre1500 = linlithgow_burials.loc[linlithgow_burials['L_DATE'] <1500]

In [ ]:
#Test your named variable by printing it out again, calling it by its name.
pre1400

So far these searches have been about the attributes of our data. Attributes are information that desribes the pologyons We can also ask questions about spatial relationships or real-world location. First, let's find the bounding box, or real world location and extents of our data. These are the smallest and largest coordinates on the x- and y- axes in the real world. We use the command 'total_bounds' to ask this question. Things 'in bounds' are inside the box.

In [ ]:
pre1400.total_bounds

In [ ]:
# Now do the same thing for the constructions. The results should be similar, but not identical.
pre1500.total_bounds

In [ ]:
# Now we will select a single burial
pre1500_325 = pre1500.loc[pre1500['FID'] == 325]
pre1500_325
pre1500_325.plot()

In [ ]:
#Now we will select another single context, SU 1178, a nice floor surface. Ok, an oddly shaped floor surface.
pre1500_201 = pre1500_201.loc[pre1500_325['FID'] == 201]
pre1500_201
pre1500_201.plot()

### One of these burials is prone, or laid out on its stomach, face down, while the other is supine, or on its back and face up.
 
Are they different shapes or sizes, or facing different directions?

In [ ]:
# To do this we have to provide a list of the values we are interested in seeing on the map, in square brackets []
pre1500.loc[pre_1500['FID'].isin([201,325])].plot(column='FID', cmap='Paired', figsize=(10, 10))

What does this map suggest? Are there any size, shape or orientation differences in these burials? What might similarities or differences mean?

#### Now we might ask a more complicated question, and look at all the burials that physically intersect with other burials.

In [ ]:
#Draw the burials from each period that intersect other burials
lpd.overlay(pre1500, pre1400, how='intersection').plot(column='AT', cmap='Accent', edgecolor='grey', figsize=(15, 15))
#You'll notice I picked a differnt colour map this time, and made the figure a bit larger.

### Now try another question. What if we wanted to find all the contexts defined as infant burials? 

Look in the 'AT' column. Infants are defined as 'INFANT (0-5)'. In many archaeological situations infants are buried differently. Perhaps they are in a separate area, or always close to an adult. These are interesting questions to investigate spatially.

In [ ]:
linlithgow_infants = linlithgow_burials[linlithgow_burials['AT'].str.contains('INFANT(0-5)')]
linlithgow_infants

In [ ]:
# Now create a map of all the burials of infants.
linlithgow_infants.plot(column='CONTEXT', cmap='Accent', edgecolor='grey', figsize=(15, 15))

### What if we wanted to know about burials that were near infant burials? Let's construct a new query.
How close is close? Let's say 0.5meters.


In [ ]:
linlithgow_burials_close = linlithgow_infants.buffer(0.5)
linlithgow_infants.plot(cmap='Accent', edgecolor='grey', figsize=(15, 15))

In [ ]:
# Now we plot the intersection between the buffered finds polygons and our construction polygons
linlithgow_infants_close.intersection(linlithgow_burials).plot(cmap='Accent', edgecolor='grey', figsize=(15, 15))

### What do we conclude? Are infants buried close to other burials? To adults?

### This ends the tutorial. You can practice writing queries (asking questions of your data) by playing around in this notebook. Try changing values or searching for different types of burials or their dates. You'll be doing this in class during your next practical!

Hopefully you learned to:
* make very simple static maps
* ask simple questions using spatial data. This is sometimes referred to as 'writing queries'.
* start thinking about the importance of spatial relationships and data in archaeology. 
